In [1]:
import sys

sys.path.append("/data/slim/models/research/slim")
%matplotlib inline

from matplotlib import pyplot as plt

import numpy as np
import os
import tensorflow as tf
import urllib2

from datasets import imagenet
from nets import vgg
from preprocessing import vgg_preprocessing
from datasets import dataset_utils

# Main slim library
from tensorflow.contrib import slim

In [2]:
input_images = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
label = tf.placeholder(tf.int64, name= "label")
summaries_dir = "/tmp/slim/tutorial"

In [3]:
def vgg16(inputs):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        net = slim.flatten(net)
        net = slim.fully_connected(net, 4096, scope='fc6')
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 4096, scope='fc7')
        net = slim.dropout(net, 0.5, scope='dropout7')
        net = slim.fully_connected(net, 2, activation_fn=None, scope='fc8')
    return net

logits = vgg16(input_images)
prob = tf.nn.softmax(logits)

In [5]:
print(logits.shape)

loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)

loss_mean = tf.reduce_mean(loss)
tf.summary.scalar('loss', loss_mean)

summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
    
    sess.run(tf.global_variables_initializer())

    x = np.ones([1, 224, 224, 3])
    summs, loss, logits_ = sess.run([summary_op, loss_mean, logits], {input_images: x, label: [1]})
    print("{}, {}".format(loss, logits_))
    
#     train_writer.add_summary(summs, 0)


(?, 2)
0.693147182465, [[  1.83889781e-09  -7.06232406e-09]]
